In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
transaction_df = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv", header = True, inferSchema =True)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format,year,col,to_timestamp


In [0]:
transaction_df.printSchema()
transaction_df.show(10, truncate = False)

root
 |-- ProductCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- ActualPrice: double (nullable = true)
 |-- Marginpercentage: integer (nullable = true)
 |-- Discountpercentage: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- TransactionAmount: double (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Mode: string (nullable = true)
 |-- InvoiceNumber: integer (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Country: string (nullable = true)

+------------------------------------+---------------------------------------------------------------------------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|ProductCode                  

In [0]:
transaction_df1=transaction_df.withColumn("Year",year(transaction_df.InvoiceDate)).withColumn("Month",month(transaction_df.InvoiceDate))

In [0]:
transaction_df1.createTempView("amazon_transaction_data")

In [0]:
transaction_df1.select("Brand","SubCategory","Month","Year").show()

+------------+-------------------+-----+----+
|       Brand|        SubCategory|Month|Year|
+------------+-------------------+-----+----+
|      landau|  all women's shoes|    1|2015|
|       adult|  all women's shoes|    1|2015|
|    woolrich|           athletic|    1|2015|
|       style|      women's shoes|    1|2015|
|         two|  all women's shoes|    1|2015|
|     michael|          necklaces|    1|2015|
|        bali|    shoes & jewelry|    1|2015|
|dr. scholl's|  all women's shoes|    1|2015|
| coloriffics|women's accessories|    1|2015|
|  timberland|       shoes & bags|    1|2015|
|      bamboo|  all women's shoes|    1|2015|
|   evolatree|women's accessories|    1|2015|
|         bar|women's accessories|    1|2015|
|     michael|  all women's shoes|    1|2015|
|      excell|   women's clothing|    1|2015|
|        vans|      women's jeans|    1|2015|
|         nwt|      women's shoes|    1|2015|
|        toms|  all women's shoes|    1|2015|
|       earth|              boots|

1 a)Find the top five performing mobile brands for the months of November, December and January r each year.

In [0]:
brand = spark.sql(" select Brand, Year, Month, sum(Quantity) as ProductSold from amazon_transaction_data where SubCategory='mobile' and Month in (11,12,1) group by Year, Month,Brand order by Year, Month,ProductSold Desc").show()


+----------+----+-----+-----------+
|     Brand|Year|Month|ProductSold|
+----------+----+-----+-----------+
|   samsung|2015|    1|        377|
|     apple|2015|    1|        223|
|     nokia|2015|    1|        168|
|       blu|2015|    1|        148|
|  motorola|2015|    1|        127|
|       htc|2015|    1|        110|
|        lg|2015|    1|        104|
|      sony|2015|    1|         91|
|blackberry|2015|    1|         81|
|    huawei|2015|    1|         50|
|    indigi|2015|    1|         45|
|  otterbox|2015|    1|         32|
|    juning|2015|    1|         29|
|  verykool|2015|    1|         23|
|   pantech|2015|    1|         22|
|    lenovo|2015|    1|         19|
|   oneplus|2015|    1|         19|
|      posh|2015|    1|         19|
|      plum|2015|    1|         18|
|       que|2015|    1|         14|
+----------+----+-----+-----------+
only showing top 20 rows



1b)Find the top five selling products for the months of November, December and January for each year.

In [0]:
sales = spark.sql("select Category, Year, Month, sum(Quantity) as ProductSold from amazon_transaction_data where  Month in (11,12,1) group by Year, Month,Category order by ProductSold, Month,Year Desc").show(50)


+-------------+----+-----+-----------+
|     Category|Year|Month|ProductSold|
+-------------+----+-----+-----------+
|    furniture|2016|   11|          3|
|    furniture|2017|   12|          3|
|apple carplay|2018|    1|          4|
|apple carplay|2016|    1|          4|
|apple carplay|2016|   11|          4|
|apple carplay|2017|   11|          8|
|        shoes|2016|   11|        539|
|        shoes|2018|    1|        551|
|        shoes|2016|   12|        642|
|        shoes|2016|    1|        650|
|        shoes|2015|   11|        666|
|        shoes|2017|    1|        712|
|        shoes|2017|   12|        793|
|        shoes|2015|   12|        884|
|        shoes|2015|    1|        895|
|        shoes|2017|   11|        992|
|  electronics|2017|   12|       2158|
|  electronics|2015|    1|       2550|
|  electronics|2016|   12|       2563|
|  electronics|2017|    1|       2619|
|  electronics|2016|    1|       2664|
|  electronics|2017|   11|       2715|
|  electronics|2018|    1

In [0]:
amazon_df1=transaction_df1.withColumn("Quarter",quarter(transaction_df1.InvoiceDate))

amazon_df1.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|Year|Month|Quarter|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|2015|    1| 

In [0]:
amazon_df1.createOrReplaceTempView("Data_Table1")

In [0]:
#2. Give valuable insights to the investors

#a. Number of products sold along with revenue aggregated quarterly

quarter_product =spark.sql("select Year,Quarter,sum(quantity) as totalbuys ,sum(TransactionAmount) as amount from Data_Table1 group by Year,Quarter order by Year,Quarter desc").show()


+----+-------+---------+-------------------+
|Year|Quarter|totalbuys|             amount|
+----+-------+---------+-------------------+
|2015|      4|    45636|       3.22182882E8|
|2015|      3|    43224|       3.17578576E8|
|2015|      2|    42069|2.23020302445285E14|
|2015|      1|    42693|       3.08548555E8|
|2016|      4|    42684|4.46040321418626E14|
|2016|      3|    44353|6.69060331721781E14|
|2016|      2|    41872|4.46040332280323E14|
|2016|      1|    49125|2.23020349928595E14|
|2017|      4|    45000|4.46040344731496E14|
|2017|      3|    40429|       2.91143349E8|
|2017|      2|    44283|       3.45020711E8|
|2017|      1|    44917|       3.23175489E8|
|2018|      2|     3905|        3.0780533E7|
|2018|      1|    42619|6.69060301392097E14|
+----+-------+---------+-------------------+



In [0]:
#b. Number of products sold in each category aggregated quarterly

quarter_category=spark.sql("select Year,Quarter,Category,sum(quantity) as totalbuys  from Data_Table1 group by Year,Quarter,Category order by Year,Quarter desc").show()

+----+-------+-------------+---------+
|Year|Quarter|     Category|totalbuys|
+----+-------+-------------+---------+
|2015|      4|     clothing|    33537|
|2015|      4|  electronics|     9917|
|2015|      4|    furniture|        3|
|2015|      4|        shoes|     2179|
|2015|      3|        shoes|     2210|
|2015|      3|  electronics|     7497|
|2015|      3|     clothing|    33517|
|2015|      2|     clothing|    32481|
|2015|      2|        shoes|     2154|
|2015|      2|  electronics|     7434|
|2015|      1|     clothing|    32683|
|2015|      1|  electronics|     7872|
|2015|      1|        shoes|     2134|
|2015|      1|apple carplay|        4|
|2016|      4|     clothing|    32097|
|2016|      4|        shoes|     1834|
|2016|      4|    furniture|        6|
|2016|      4|  electronics|     8743|
|2016|      4|apple carplay|        4|
|2016|      3|    furniture|        6|
+----+-------+-------------+---------+
only showing top 20 rows



In [0]:
#c A number of products sold for each brand aggregated quarterly.

quarter_brand = spark.sql("select Brand,year,quarter,sum(Quantity) as Num_Product_Sold from Data_Table1 group by year,Brand,quarter order by Num_Product_Sold desc").show()

+-------+----+-------+----------------+
|  Brand|year|quarter|Num_Product_Sold|
+-------+----+-------+----------------+
|   nike|2016|      1|            2902|
|   nike|2017|      1|            2743|
|   nike|2017|      2|            2731|
|   nike|2016|      4|            2729|
|   nike|2017|      4|            2645|
|   nike|2015|      1|            2642|
|   nike|2018|      1|            2623|
|   nike|2016|      3|            2544|
|   nike|2015|      3|            2520|
|   nike|2015|      4|            2520|
|   nike|2016|      2|            2399|
|   nike|2015|      2|            2336|
|   nike|2017|      3|            2330|
|samsung|2015|      4|            1536|
|samsung|2017|      2|            1463|
|samsung|2016|      3|            1434|
|samsung|2017|      3|            1349|
|samsung|2016|      4|            1334|
|samsung|2017|      1|            1302|
|samsung|2016|      1|            1237|
+-------+----+-------+----------------+
only showing top 20 rows



In [0]:
# 3Find products that generated profit
# a Top 5 profit margin generating products for a given month.

spark.sql("select Brand,SubCategory,(Marginpercentage-Discountpercentage) as profit,Month,year from amazon_transaction_data group by year,SubCategory,Brand,Month,profit order by profit desc").show()

+-------------+--------------------+------+-----+----+
|        Brand|         SubCategory|profit|Month|year|
+-------------+--------------------+------+-----+----+
|      express|   all women's shoes|    20|    2|2015|
|         geox|     all men's shoes|    20|    3|2015|
|        sorel|     shoes & jewelry|    20|    2|2015|
|          rdx|         men's shoes|    20|    1|2015|
|         nike|   all women's shoes|    20|    3|2015|
|         reef|         men's shoes|    20|    4|2015|
|    dreamgirl|   all women's shoes|    20|    3|2015|
|       vision|               jeans|    20|    5|2015|
|          blu|              mobile|    20|    3|2015|
|       azzuro|         men's boots|    20|    1|2015|
|bcbgeneration|other wholesale w...|    20|    3|2015|
|       deluxe|       women's shoes|    20|    5|2015|
|     original|         men's shoes|    20|    6|2015|
|     academie|         hats & caps|    20|    1|2015|
|        crocs|    petite swimsuits|    20|    6|2015|
|       ja

In [0]:
#4 Brands that need marketing
#a Brand wise segregation of best performing categories profits wise
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from amazon_transaction_data group by Brand,Category,profit order by profit desc").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|               peter|   clothing|    20|
|             kenwood|electronics|    20|
|            castelli|   clothing|    20|
|                 dvs|   clothing|    20|
|               annie|   clothing|    20|
|              saddle|   clothing|    20|
|          underwraps|   clothing|    20|
|           haflinger|   clothing|    20|
|                 jbu|   clothing|    20|
|            muckboot|   clothing|    20|
|            siriusxm|electronics|    20|
|            funtasma|   clothing|    20|
|               perry|   clothing|    20|
|               donna|      shoes|    20|
|              noriko|   clothing|    20|
|                tumi|electronics|    20|
|                 cet|   clothing|    20|
|               nicki|      shoes|    20|
|yu&yu soccer men'...|   clothing|    20|
|               saint|   clothing|    20|
+--------------------+-----------+

In [0]:
# b Brand wise segregation of least performing categories orders wise
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from amazon_transaction_data group by Brand,Category,profit order by profit asc").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|           alexander|   clothing|   -50|
|               ridge|   clothing|   -50|
|               bgood|   clothing|   -50|
|             pantech|electronics|   -50|
|            alliance|   clothing|   -50|
|             bellini|   clothing|   -50|
|             natural|   clothing|   -50|
|         ellie shoes|   clothing|   -50|
|                five|   clothing|   -50|
|fruit of the loom...|   clothing|   -50|
|                easy|   clothing|   -50|
|              report|   clothing|   -50|
|                lowa|   clothing|   -50|
|                 nwt|   clothing|   -50|
|             emporio|   clothing|   -50|
|            siriusxm|electronics|   -50|
|            onitsuka|   clothing|   -50|
|             dickies|   clothing|   -50|
|               bally|   clothing|   -50|
|                moda|   clothing|   -50|
+--------------------+-----------+

In [0]:
# 5Find the products that should be part of the lighting deals.
# a Find the top five selling products in a given month
spark.sql("select SubCategory,Month,year,sum(Quantity) as Total_sold from amazon_transaction_data group by SubCategory,Month,year order by Total_sold desc").show()

+-----------------+-----+----+----------+
|      SubCategory|Month|year|Total_sold|
+-----------------+-----+----+----------+
|all women's shoes|    2|2016|      3345|
|all women's shoes|   11|2017|      3260|
|           mobile|    3|2016|      3030|
|all women's shoes|    5|2017|      3015|
|all women's shoes|   12|2015|      2970|
|           mobile|   12|2015|      2935|
|all women's shoes|    8|2016|      2925|
|all women's shoes|    1|2015|      2891|
|all women's shoes|    3|2018|      2846|
|           mobile|    8|2017|      2787|
|           mobile|   10|2015|      2738|
|           mobile|    8|2016|      2727|
|           mobile|   11|2015|      2701|
|all women's shoes|    3|2016|      2660|
|      men's shoes|   11|2017|      2659|
|all women's shoes|    8|2017|      2646|
|all women's shoes|    2|2017|      2600|
|all women's shoes|   12|2017|      2596|
|all women's shoes|    5|2015|      2593|
|all women's shoes|    9|2015|      2587|
+-----------------+-----+----+----

In [0]:
# b Find the top five products having the highest margin in a given month
spark.sql("select SubCategory,Marginpercentage,Month,Year from amazon_transaction_data group by SubCategory,Marginpercentage,Month,Year order by Marginpercentage desc").show()



+--------------------+----------------+-----+----+
|         SubCategory|Marginpercentage|Month|Year|
+--------------------+----------------+-----+----+
|see more under ar...|              30|    4|2015|
|             walking|              30|    2|2015|
|     shoes & jewelry|              30|    5|2015|
|      women's skirts|              30|    4|2015|
|           moccasins|              30|    5|2015|
|      men's clothing|              30|    4|2015|
|see more samsung ...|              30|    5|2015|
|women's coats & j...|              30|    1|2015|
|               women|              30|    6|2015|
|            slippers|              30|    4|2015|
|             apparel|              30|    6|2015|
|  women's sunglasses|              30|    3|2015|
|              skirts|              30|    2|2015|
|       lawn & garden|              30|    2|2015|
|               youth|              30|    3|2015|
|                bras|              30|    5|2015|
|women's anniversa...|         

In [0]:
# c Find the five lowest selling products in a given month.
spark.sql("select Category,Quantity,Month,Year from amazon_transaction_data group by Category,Quantity,Month,Year order by Quantity ").show()

+-----------+--------+-----+----+
|   Category|Quantity|Month|Year|
+-----------+--------+-----+----+
|      shoes|       1|    1|2015|
|electronics|       1|   10|2015|
|   clothing|       1|    1|2015|
|   clothing|       1|    4|2015|
|   clothing|       1|    2|2015|
|      shoes|       1|    4|2015|
|      shoes|       1|    3|2015|
|      shoes|       1|    6|2015|
|electronics|       1|    1|2015|
|   clothing|       1|    3|2015|
|electronics|       1|    4|2015|
|   clothing|       1|    5|2015|
|   clothing|       1|   11|2015|
|electronics|       1|    5|2015|
|electronics|       1|   11|2015|
|electronics|       1|    8|2015|
|   clothing|       1|    9|2015|
|electronics|       1|    7|2015|
|      shoes|       1|    2|2015|
|   clothing|       1|    8|2015|
+-----------+--------+-----+----+
only showing top 20 rows

